# Extraction of features from physiologic signals in polysomnograph data

Database source: 
    
    Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23):e215-e220 [Circulation Electronic Pages; http://circ.ahajournals.org/content/101/23/e215.full]; 2000 (June 13).


In [12]:
from __future__ import division
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as ss
import seaborn as sns
from scipy.ndimage import convolve1d
from scipy.stats import entropy
import itertools
import sleep_utils as su

import pickle
import datetime  
import time 
import wfdb
import glob


from biosppy.signals import ecg
import nolds


%matplotlib inline
sns.set()
pd.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load a file from database

In [3]:
data_dir = '../data/PSG_database/'

co2_sig = []
ppg_sig = []

filenames = []

sleep_data_sigs = {}
sleep_data_fields = {}

data_keys = []
for fname in glob.iglob(data_dir + '/*.dat'):#, recursive=True):
    # filename manipulation
    base = os.path.basename(fname)
    key = os.path.splitext(base)[0]
    data_keys.append(key)
    
    # load data
    sig, fields=wfdb.rdsamp(data_dir+key)
    sleep_data_sigs[key] = sig
    sleep_data_fields[key] = fields

In [4]:
ann_index = {}
ann_labels = {}

set_stages = set()
for key in data_keys:
    annotation = wfdb.rdann(data_dir + key, 'st')
    ann_index_sig = annotation[0]
    ann_label_sig = annotation[5]
    label_filt = [(x, y) for x, y in zip(ann_index_sig, ann_label_sig) if x >1 ]
    unzipped = zip(*label_filt)
    ann_index[key] = unzipped[0]
    ann_labels[key] = unzipped[1]


In [5]:
resp_data_set = {}
ecg_data_set = {}
bp_data_set = {}
time_data_set = {}
annot_data_set = {}


fs = 250
dec_prec = len(str(1/fs).split('.')[1])

for key in data_keys:
    
    resp_data_set[key] = (sleep_data_sigs[key])[:, (sleep_data_fields[key])['signame'].index('Resp')]
    ecg_data_set[key] = (sleep_data_sigs[key])[:, (sleep_data_fields[key])['signame'].index('ECG')]
    bp_data_set[key] = (sleep_data_sigs[key])[:, (sleep_data_fields[key])['signame'].index('BP')]
    time_data_set[key] = np.arange(0, len(resp_data_set[key]))/fs

    # Note: Only the first character of the annotation was taken as label
    # Example: 1, 2, 3, 4, R, W, M
    stages_list = [x.rsplit(' ')[0][0] for x in ann_labels[key]]

    annot_data_set[key] = stages_list


In [6]:
features_dir = data_dir + "PSG_features/"

In [7]:
# pickle.dump( resp_data_set, open( features_dir + "resp_data_set.pkl", "wb" ))
# pickle.dump( ecg_data_set, open( features_dir + "ecg_data_set.pkl", "wb" ))
# pickle.dump( bp_data_set, open( features_dir + "bp_data_set.pkl", "wb" ))
# pickle.dump( time_data_set, open( features_dir + "time_data_set.pkl", "wb" ))
# pickle.dump( annot_data_set, open( features_dir + "annot_data_set.pkl", "wb" ))

#### *Annotations*

Stages:
   * W: subject is awake
   * 1: sleep stage 1
   * 2: sleep stage 2
   * 3: sleep stage 3
   * 4: sleep stage 4
   * R: REM sleep
   
Other descriptions:
   * H: Hypopnea
   * HA: Hypopnea with arousal
   * OA: Obstructive apnea
   * X: Obstructive apnea with arousal
   * CA: Central apnea
   * CAA: Central apnea with arousal
   * L: Leg movements
   * LA: Leg movements with arousal
   * A: Unspecified arousal
   * MT: Movement time

## Get Features

### I. Previous sleep stage

Note: Get features for every window of data

In [8]:
win_dur = 30 # duration of window, unit: seconds, size of one epoch 
win_size = fs*win_dur # length of window
win_step = 1 # duration by which the window slides, unit: seconds
win_int = win_step*fs # length by which the window slides

In [14]:
feature_prev_stage = {}
feature_prev_stage_light = {}
feature_prev_stage_deep = {}
feature_prev_stage_rem = {}
feature_prev_stage_wake = {}
feature_prev_stage_move = {}

for key in data_keys:
    previous = annot_data_set[key][1:]
    prev_stage = [np.nan] + previous
    feature_prev_stage[key] = {'_feat' : prev_stage}
    
    # NREM-light or not
    feature_prev_stage_light[key] = {'_feat' : [np.nan] +map(lambda x: 0 if x not in ['1', '2'] else 1, previous)}
    
    # NREM-deep or not
    feature_prev_stage_deep[key] = {'_feat' : [np.nan] +map(lambda x: 0 if x not in ['1', '2'] else 1, previous)}
    
    # REM or not
    feature_prev_stage_rem[key] = {'_feat' : [np.nan] + map(lambda x: 0 if x!= 'R' else 1,previous)}
    
    # Wake or not
    feature_prev_stage_wake[key] = {'_feat' :[np.nan] + map(lambda x: 0 if x!= 'W' else 1,previous)}
    
    # Wake or not
    feature_prev_stage_move[key] = {'_feat' :[np.nan] + map(lambda x: 0 if x!= 'M' else 1,previous)}


In [15]:
# pickle.dump( feature_prev_stage, open( features_dir + "feature_prev_stage.pkl", "wb" ))
# pickle.dump( feature_prev_stage_light, open( features_dir + "feature_prev_stage_light.pkl", "wb" ))
# pickle.dump( feature_prev_stage_deep, open( features_dir + "feature_prev_stage_deep.pkl", "wb" ))
# pickle.dump( feature_prev_stage_rem, open( features_dir + "feature_prev_stage_rem.pkl", "wb" ))
# pickle.dump( feature_prev_stage_wake, open( features_dir + "feature_prev_stage_wake.pkl", "wb" ))
# pickle.dump( feature_prev_stage_move, open( features_dir + "feature_prev_stage_move.pkl", "wb" ))


### II. Respiration signal

In [9]:
# Respiratory frequency range 
min_normrange = 4/60 # unit: cycles per second
max_normrange = 65/60 # unit: cycles per second

resp_data_set_epochs = {}

# Divide ECG data per 30-second epoch. 

for key in data_keys:
    resp_sig = resp_data_set[key]
    fft_resp = np.fft.fft(resp_sig)
    fft_freqs = np.fft.fftfreq(len(resp_sig), 1/fs)
    
    # Remove frequencies which are outside the expected range
    fft_resp[abs(fft_freqs) < min_normrange] = 0
    fft_resp[abs(fft_freqs) > max_normrange] = 0
    
    resp_filt_sig = np.real(np.fft.ifft(fft_resp))
    time_sig = np.arange(len(resp_sig))/fs

    # Divide into windows
    resp_windows = su.divide_to_epochs(resp_filt_sig, ann_index[key], win_dur, fs)
    time_windows = su.divide_to_epochs(time_sig, ann_index[key], win_dur, fs)
    
    resp_data_set_epochs[key] = pd.DataFrame({'resp': list(resp_windows), '_time': list(time_windows)})


#### II.1 Respiration rate
*Frequency corresponding to the highest peak in the epoch's power spectrum (unit: breaths/minute)*

In [19]:
feature_resp_rate = {}
# feature_resp_rate_fd = {}
# feature_resp_rate_sd = {}

for key in data_keys:
    resp_data = resp_data_set_epochs[key]
    resp_data['power_spectrum'] = resp_data.resp.apply(lambda x: ss.periodogram(x, fs=fs))
    
    # Compute respiration rate
    resp_data['resp_rate'] = resp_data.power_spectrum.apply(lambda x: (x[0])[np.argmax(x[1])]*60)

    feature_resp_rate[key] = {'_feat' : resp_data.resp_rate.values}


In [20]:
# pickle.dump( feature_resp_rate, open( features_dir + "feature_resp_rate.pkl", "wb" ))

#### II.2 Histogram of Respiration data
Use the counts for each bin of the magnitude histogram.

In [21]:
nbins = 5

feature_hist_resp = {}

for key in data_keys:
#     print("Computing for "+ key + "...")
    resp_data = resp_data_set_epochs[key]

    hist_windows = resp_data.resp.apply(lambda x: np.histogram(x, bins=nbins, normed=True)[0])
    feature_hist_resp[key] = {'_feat' : (np.asarray(hist_windows))}
    
pickle.dump( feature_hist_resp, open( features_dir + "feature_hist_resp.pkl", "wb" ))    

#### II.3 Ratio of standard deviation and mean of magnitude

In [10]:
feature_mean_vs_std_resp = {}

for key in data_keys:
#     print("Computing for "+ key + "...")
    resp_data = resp_data_set_epochs[key]

    mean_vs_std = resp_data.resp.apply(lambda x: (np.mean(x)/np.std(x)))
    feature_mean_vs_std_resp[key] = {'_feat' : list(mean_vs_std)}

#### II.4 Ratio of  mean and standard deviation  of peak-to-peak duration of respiration signal

In [11]:
feature_mean_vs_std_p2p_dur_resp = {}
# feature_max_vs_std_p2p_dur_resp = {}
# feature_min_vs_std_p2p_dur_resp = {}

for key in data_keys:
    print("Computing for "+ key + "...")
    resp_data = resp_data_set_epochs[key]

    p2p_dur = resp_data.resp.apply(lambda x: su.bio_signal_peak_detect(x, fs, 'resp'))\
                            .apply(lambda x: np.array(x[0][1:]) - np.array(x[0][:-1]))
    mean_vs_std = p2p_dur.apply(lambda x: np.nanmean(x)/np.nanstd(x))
    feature_mean_vs_std_p2p_dur_resp[key] = {'_feat' : list(mean_vs_std)}           
    
#### II.5 Ratio of max and mean of magnitude of peak-to-peak duration
#### II.6 Ratio of min and mean of magnitude of peak-to-peak duration
    
#     max_vs_std = p2p_dur.apply(lambda x: np.nanmax(x)/np.nanmean(x))
#     feature_max_vs_mean_p2p_dur_resp[key] = {'_feat' : list(max_vs_std)}           
    
#     min_vs_std = p2p_dur.apply(lambda x: np.nanmin(x)/np.nanmean(x))
#     feature_min_vs_mean_p2p_dur_resp[key] = {'_feat' : list(min_vs_std)}           


Computing for slp01a...


peakdetect.py:200: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  for index, (x, y) in enumerate(zip(x_axis[:-lookahead],
peakdetect.py:201: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y_axis[:-lookahead])):
peakdetect.py:213: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if y_axis[index:index+lookahead].max() < mx:
peakdetect.py:231: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if y_axis[index:index+lookahead].min() > mn:


Computing for slp01b...
Computing for slp02a...
Computing for slp02b...
Computing for slp03...
Computing for slp04...
Computing for slp14...
Computing for slp16...
Computing for slp32...


/Users/eventura/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:675: RuntimeWarning: Mean of empty slice
  warnings.warn("Mean of empty slice", RuntimeWarning)
/Users/eventura/anaconda/lib/python2.7/site-packages/numpy/lib/nanfunctions.py:1147: RuntimeWarning: Degrees of freedom <= 0 for slice.
  warnings.warn("Degrees of freedom <= 0 for slice.", RuntimeWarning)


Computing for slp37...
Computing for slp41...
Computing for slp45...
Computing for slp48...
Computing for slp59...
Computing for slp60...
Computing for slp61...
Computing for slp66...
Computing for slp67x...


#### II.5 Ratio of mean and standard deviation of amplitude
#### II.6 Ratio of max and mean of magnitude of amplitude
#### II.7 Ratio of min and mean of magnitude of amplitude
 

In [16]:
feature_mean_vs_std_amp_resp = {}
feature_max_vs_mean_amp_resp = {}
feature_min_vs_mean_amp_resp = {}

for key in data_keys:
    print("Computing for "+ key + "...")
    resp_data = resp_data_set_epochs[key]

    amplitude = resp_data.resp.apply(lambda x: su.compute_amplitude(x, fs, 'resp'))
    mean_vs_std = amplitude.apply(lambda x: np.nanmean(x)/np.nanstd(x))
    feature_mean_vs_std_amp_resp[key] = {'_feat' : list(mean_vs_std)}           
    
    max_vs_mean = amplitude.apply(lambda x: np.nanmax(x)/np.nanmean(x))
    feature_max_vs_mean_amp_resp[key] = {'_feat' : list(max_vs_mean)}           
    
    min_vs_mean = amplitude.apply(lambda x: np.nanmax(x)/np.nanmean(x))
    feature_min_vs_mean_amp_resp[key] = {'_feat' : list(min_vs_mean)}    

Computing for slp01a...


peakdetect.py:200: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  for index, (x, y) in enumerate(zip(x_axis[:-lookahead],
peakdetect.py:201: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  y_axis[:-lookahead])):
peakdetect.py:213: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if y_axis[index:index+lookahead].max() < mx:
peakdetect.py:231: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if y_axis[index:index+lookahead].min() > mn:


Computing for slp01b...
Computing for slp02a...


KeyboardInterrupt: 

#### II.8 Sample entropy of manitude of respiration signal


In [107]:
feature_sampen_resp = {}


for key in data_keys:
    print("Computing for "+ key + "...")
    resp_data = resp_data_set_epochs[key]

    sampen_val = resp_data.resp.apply(lambda x: (nolds.sampen(x)))
    feature_sampen_resp[key] = {'_feat' : list(sampen_val)}           
    

Computing for slp01a...
Computing for slp01b...
Computing for slp02a...
Computing for slp02b...
Computing for slp03...
Computing for slp04...
Computing for slp14...
Computing for slp16...
Computing for slp32...
Computing for slp37...
Computing for slp41...
Computing for slp45...
Computing for slp48...
Computing for slp59...
Computing for slp60...
Computing for slp61...
Computing for slp66...
Computing for slp67x...


In [ ]:
### AR and MA parameters

In [33]:
sample_sig = resp_data.resp.ix[0]
sample_time = np.arange(len(sample_sig))

In [42]:
sample_timestamp = np.zeros_like(sample_time)

for i in sample_time:
    sample_timestamp[i] = datetime.datetime(i)

TypeError: Required argument 'month' (pos 2) not found

In [38]:
import statsmodels.api as sm
arma_mod30 = sm.tsa.ARMA(sample_sig, (3,0))

In [41]:
arma_mod30.fit()

ValueError: The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.

In [36]:
arma_model = arima.ARIMA
arma_model.fit(resp_data.resp.ix[0])

TypeError: unbound method fit() must be called with ARIMA instance as first argument (got ndarray instance instead)

### III. ECG signal

In [10]:
# Divide ECG data per 30-second epoch. 

ecg_data_set_epochs = {}

for key in data_keys:
    ecg_sig = ecg_data_set[key]
    time_sig = np.arange(len(ecg_sig))/fs

    # Divide into windows
    ecg_windows = su.divide_to_epochs(ecg_sig, ann_index[key], win_dur, fs)
    time_windows = su.divide_to_epochs(time_sig, ann_index[key], win_dur, fs)
    
    ecg_data_set_epochs[key] = pd.DataFrame({'ecg': list(ecg_windows), '_time': list(time_windows)})

#### III.1 Heart rate
*Reciprocal of the mean R-R interval in an epoch (unit: beats/minute)*


#### III.2 Heart rate variability
*Standard deviation of the R-R intervals in an epoch (unit: milliseconds)*


In [14]:
feature_heart_rate = {}
feature_heart_rate_var = {}

for key in data_keys:
    ecg_data = ecg_data_set_epochs[key]
    
    # Get time corresponding to R peaks
    ecg_data['r_peaks'] = ecg_data.ecg.apply(lambda x: ecg.hamilton_segmenter(x, sampling_rate=250)['rpeaks'])
    ecg_data['r_time'] = ecg_data.apply(lambda x: list(x._time[x.r_peaks]), axis = 1)
    
    # Compute heart rate
    ecg_data['heart_rate'] = ecg_data.r_time.apply(lambda x: su.heart_rate(np.array(x))*60)
    ecg_data['window_time'] = ecg_data['_time'].apply(lambda x: x[-1])
        
    # Compute heart rate variability
    ecg_data['heart_rate_var'] = ecg_data.r_time.apply(lambda x: su.heart_rate_var(np.array(x))*1000)
    
    feature_heart_rate[key] = {'_feat' : ecg_data.heart_rate.values}
    feature_heart_rate_var[key] = {'_feat' : ecg_data.heart_rate_var.values}

#### III.3 Histogram of ECG data
Use the counts for each bin of the magnitude histogram.

In [15]:
feature_hist_ecg = {}

for key in data_keys: 
#     print("Computing for "+ key + "...")
    ecg_data = ecg_data_set_epochs[key]

    hist_windows = ecg_data.ecg.apply(lambda x: np.histogram(x, bins=nbins, normed=True)[0])
    feature_hist_ecg[key] = {'_feat' : (np.asarray(hist_windows))}

#### III.4 Ratio of standard deviation and mean of magnitude

In [16]:
feature_mean_vs_std_ecg = {}

for key in data_keys:
#     print("Computing for "+ key + "...")
    ecg_data = ecg_data_set_epochs[key]

    mean_vs_std = ecg_data.ecg.apply(lambda x: (np.mean(x)/np.std(x)))
    feature_mean_vs_std_ecg[key] = {'_feat' : list(mean_vs_std)}

#### III.5 Ratio of  mean and standard deviation  of peak-to-peak duration of ecg signal

In [17]:
feature_mean_vs_std_p2p_dur_ecg = {}

for key in data_keys:
    print("Computing for "+ key + "...")
    ecg_data = ecg_data_set_epochs[key]

    p2p_dur = ecg_data.ecg.apply(lambda x: su.bio_signal_peak_detect(x, fs, 'ecg'))\
                            .apply(lambda x: np.array(x[0][1:]) - np.array(x[0][:-1]))
    mean_vs_std = p2p_dur.apply(lambda x: np.nanmean(x)/np.nanstd(x))
    feature_mean_vs_std_p2p_dur_ecg[key] = {'_feat' : list(mean_vs_std)}           
    


Computing for slp01a...
Computing for slp01b...
Computing for slp02a...
Computing for slp02b...
Computing for slp03...
Computing for slp04...
Computing for slp14...
Computing for slp16...
Computing for slp32...
Computing for slp37...
Computing for slp41...
Computing for slp45...
Computing for slp48...
Computing for slp59...
Computing for slp60...
Computing for slp61...
Computing for slp66...
Computing for slp67x...


#### III.6 Ratio of mean and standard deviation of amplitude
#### III.7 Ratio of max and mean of magnitude of amplitude
#### III.8 Ratio of min and mean of magnitude of amplitude
 

In [18]:
feature_mean_vs_std_amp_ecg = {}
feature_max_vs_mean_amp_ecg = {}
feature_min_vs_mean_amp_ecg = {}

for key in data_keys:
    print("Computing for "+ key + "...")
    ecg_data = ecg_data_set_epochs[key]

    amplitude = ecg_data.ecg.apply(lambda x: su.compute_amplitude(x, fs, 'ecg'))
    mean_vs_std = amplitude.apply(lambda x: np.nanmean(x)/np.nanstd(x))
    feature_mean_vs_std_amp_ecg[key] = {'_feat' : list(mean_vs_std)}           
    
    max_vs_mean = amplitude.apply(lambda x: np.nanmax(x)/np.nanmean(x))
    feature_max_vs_mean_amp_ecg[key] = {'_feat' : list(max_vs_mean)}           
    
    min_vs_mean = amplitude.apply(lambda x: np.nanmax(x)/np.nanmean(x))
    feature_min_vs_mean_amp_ecg[key] = {'_feat' : list(min_vs_mean)}    

Computing for slp01a...
Computing for slp01b...
Computing for slp02a...
Computing for slp02b...
Computing for slp03...
Computing for slp04...
Computing for slp14...
Computing for slp16...
Computing for slp32...
Computing for slp37...
Computing for slp41...
Computing for slp45...
Computing for slp48...
Computing for slp59...
Computing for slp60...
Computing for slp61...
Computing for slp66...
Computing for slp67x...


### IV. Blood Pressure signal

In [11]:
# Divide ECG data per 30-second epoch. 

bp_data_set_epochs = {}

for key in data_keys:
    bp_sig = bp_data_set[key]
    time_sig = np.arange(len(bp_sig))/fs

    # Divide into windows
    bp_windows = su.divide_to_epochs(bp_sig, ann_index[key], win_dur, fs)
    time_windows = su.divide_to_epochs(time_sig, ann_index[key], win_dur, fs)
    
    bp_data_set_epochs[key] = pd.DataFrame({'bp': list(bp_windows), '_time': list(time_windows)})


#### IV.1 Histogram of BP data
Use the counts for each bin of the magnitude histogram.

In [20]:
feature_hist_bp = {}

for key in data_keys:
#     print("Computing for "+ key + "...")
    bp_data = bp_data_set_epochs[key]

    hist_windows = bp_data.bp.apply(lambda x: np.histogram(x, bins=nbins, normed=True)[0])
    feature_hist_bp[key] = {'_feat' : (np.asarray(hist_windows))}

#### IV.2 Ratio of standard deviation and mean of magnitude

In [21]:
feature_mean_vs_std_bp = {}

for key in data_keys:
#     print("Computing for "+ key + "...")
    bp_data = bp_data_set_epochs[key]

    mean_vs_std = bp_data.bp.apply(lambda x: (np.mean(x)/np.std(x)))
    feature_mean_vs_std_bp[key] = {'_feat' : list(mean_vs_std)}

In [75]:
# #raw value
# feature_raw_bp = {}

# for key in data_keys:
# #     print("Computing for "+ key + "...")
#     bp_data = bp_data_set_epochs[key]

#     raw_val = bp_data.bp.apply(lambda x: x[-1])
#     feature_raw_bp[key] = {'_feat' : list(raw_val)}